In [ ]:
# notebook settings
%load_ext autoreload
%autoreload 2
# %matplotlib notebook

In [ ]:
# others
import numpy as np

# drake
from pydrake.geometry import SceneGraph
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import UniformGravityFieldElement
from pydrake.multibody.plant import MultibodyPlant
from pydrake.multibody.parsing import Parser
from pydrake.systems.framework import DiagramBuilder, BasicVector
from pydrake.systems.analysis import Simulator
from pydrake.systems.meshcat_visualizer import MeshcatVisualizer
from pydrake.systems.drawing import plot_system_graphviz

# hand-written controller
from controller import Controller

In [ ]:
builder = DiagramBuilder()
pusher_slider, scene_graph = AddMultibodyPlantSceneGraph(
    builder,
    MultibodyPlant(time_step=0.0003)
)
parser = Parser(plant=pusher_slider)
parser.AddModelFromFile("ground.urdf")
parser.AddModelFromFile("pusher.urdf")
parser.AddModelFromFile("slider.urdf")
# gravity = UniformGravityFieldElement(np.array([0,0,-9.81]))
# pusher_slider.AddForceElement(gravity)
ground = pusher_slider.GetFrameByName('ground_link')
pusher_origin = pusher_slider.GetFrameByName('pusher_x')
world = pusher_slider.world_frame()
pusher_slider.WeldFrames(world, ground)
pusher_slider.WeldFrames(world, pusher_origin)
pusher_slider.Finalize()

In [ ]:
controller = builder.AddSystem(Controller(pusher_slider))
builder.Connect(
    pusher_slider.get_state_output_port(),
    controller.get_input_port(0)
)
builder.Connect(
    controller.get_output_port(0),
    pusher_slider.get_actuation_input_port()
)

In [ ]:
# first, from terminal: meshcat-server
visualizer = builder.AddSystem(
    MeshcatVisualizer(scene_graph)
)
builder.Connect(
    scene_graph.get_pose_bundle_output_port(),
    visualizer.get_input_port(0)
)
diagram = builder.Build()

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 300
plot_system_graphviz(diagram)
plt.show()

In [ ]:
for i in range(10,20):
    print(i)
    np.random.seed(i)
    diagram_context = diagram.CreateDefaultContext()
    pusher_slider_context = diagram.GetMutableSubsystemContext(pusher_slider, diagram_context)
    l = 1.5
    x, y = np.random.rand(2)*2*l - l*np.ones(2)
    theta = np.random.rand(1)*2*np.pi
    q0 = np.array([
        np.cos(theta/2), 0, 0, np.sin(theta/2), # quaternions slider
        x, y, .1, # translation slider
        2.5, 2.5 # translation pusher
    ])
    v0 = np.array([0,0,0,0,0,0,0,0])
    x0 = np.concatenate((q0, v0))
    pusher_slider.SetPositionsAndVelocities(pusher_slider_context, x0)

    simulator = Simulator(diagram, diagram_context)
    simulator.set_target_realtime_rate(1.0)
    # simulator.set_publish_every_time_step(False)
    simulator.Initialize()
    simulator.StepTo(5)